# Search Cursor with spatial filter

This script uses the spatial filter and spatial relationship parameters, new to ArcGIS Pro 3.2

In [25]:
import arcpy

In [26]:
ROW = r"C:\Users\tim10393\Downloads\PADA_2.0_Oct20_Demos\DemoData\PADA\ParcelFabricsInApps\Polk_Initial_Lesson_8_Solution.gdb\ParcelEntry\ROW"
lot = r"C:\Users\tim10393\Downloads\PADA_2.0_Oct20_Demos\DemoData\PADA\ParcelFabricsInApps\Polk_Initial_Lesson_8_Solution.gdb\ParcelEntry\Lot"
outputCSV = r"C:\Users\tim10393\Downloads\AssessmentParcels.csv"

First the script selects the Right-of-Ways of interest, and appends the OID and geometry object for each record into a list.

In [27]:
exp = "OBJECTID = 1 Or OBJECTID = 2"

selROW = arcpy.management.SelectLayerByAttribute(in_layer_or_view=ROW,where_clause=exp)
ROWValues = []

selROWCur = arcpy.da.SearchCursor(selROW, ["OID@", "Shape@"])
for row in selROWCur:
    print(row)
    ROWValues.append(row)
del selROWCur

(1, <Polygon object at 0x1dc2ea3e190[0x1dc2ea3e120]>)
(2, <Polygon object at 0x1dc2ec543a0[0x1dc2ec54ab0]>)


The script will return the information for lots within 500 feet of the selected right of ways. The spatial relationship parameter has a subset of possible relationship types. There is not a parameter for searching within a 500 foot distance. So, we will put a 500 foot buffer around a geometry object and set the spatial relationship value to intersects.

We create the SearchCursor in a for loop since we are processing multiple selected ROWs.

In [28]:
parcelList = ["ROW OID, Lot OID, Lot Name, Lot Area"]

for rowVal in ROWValues:
    ROWoid = rowVal[0]
    ROWgeom = rowVal[1]
    print(f"Processing ROW: {rowVal[0]}")
    spCursor = arcpy.da.SearchCursor(lot, ["OID@", "Name", "Shape_Area"],
                                spatial_filter=ROWgeom.buffer(500),
                                spatial_relationship="INTERSECTS")
    i = 0
    for spRow in spCursor:
        print(spRow)
        parcelList.append(f"{ROWoid},{spRow[0]},{spRow[1]},{spRow[2]}")
        i += 1
    print(f"There are {i} lots within 500 feet\n")
    del spCursor

Processing ROW: 1
(32, '19', 9794.763128139197)
(34, '20', 28825.447986742416)
(35, '21', 12847.732913202964)
(36, '18', 6603.234301032638)
(37, '17', 6212.109477652729)
(38, '22', 13461.752557903284)
(39, '16', 7375.493209652511)
(40, '23', 7632.622731026843)
(41, '24', 16793.742006224253)
(43, '15', 9495.30410908653)
(44, '25', 18799.902236410835)
(45, '26', 9871.937154217472)
(47, '09', 7474.807330550021)
(48, '14', 8608.394286603303)
(49, '10', 9048.222100065368)
(50, '08', 8009.869898739259)
(51, '11', 6873.312234422231)
(53, '12', 6997.071309368926)
(63, None, 29.998490956769267)
(64, None, 44.26424912793014)
(65, '29', 7349.788983468878)
(66, '31', 6902.55099960014)
(67, '30', 8028.2376538808585)
(68, '32', 20328.84933406458)
(69, '28', 7341.430504070682)
(70, '27', 7823.607036680804)
(71, '33', 20229.066607797075)
(72, '34', 7697.594557589343)
(73, '35', 8125.808482950045)
(85, '24B', 24426.364737251097)
There are 30 lots within 500 feet

Processing ROW: 2
(32, '19', 9794.76312

Lastly, we will save the returned information as a CSV file.

In [29]:
textBody = '\n'.join(parcelList)
csvFile = open(outputCSV, "w")
csvFile.write(textBody)
csvFile.close()
print("Script Complete")

Script Complete
